In [1]:
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import math
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import pandas as pd
import math

import os

from einops.layers.torch import Rearrange, Reduce

import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

os.environ["WANDB_API_KEY"] = "KEY"
os.environ["WANDB_MODE"] = 'offline'
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import tqdm

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset
import random
from torch import Tensor
from torch import nn
from torch.optim import AdamW

- load data

In [2]:
class EP_dataset(Dataset):
    def __init__(self, EP_folderpath, image_folderpath, batch_size=20):
        self.EP_folderpath = EP_folderpath
        self.image_folderpath = image_folderpath
        self.batch_size = batch_size
        self.progress_counter = 0
        self.max_pool = nn.MaxPool1d(kernel_size=5, stride=5)

        self.EP_data_list, self.image_label_list = self.EP_dataset()

    def __len__(self):
        return len(self.EP_data_list)

    def __getitem__(self, idx):
        EP_data = self.EP_data_list[idx].T
        print(EP_data.shape)
        label = self.image_label_list[idx]

        EP_data = torch.tensor(EP_data, dtype=torch.float32)
        #EP_data = self.max_pool(EP_data)
        
        label = torch.tensor(int(label), dtype=torch.long)  


        self.progress_counter += 1
        if self.progress_counter % 1000 == 0:
            print(f"Processed {self.progress_counter}/{len(self.EP_data_list)} items")

        return EP_data, label


    def EP_dataset(self):
        EP_segmentation_list = os.listdir(self.EP_folderpath)
        print(f'Total Dataset Number: {len(EP_segmentation_list)}')
        EP_data_list = []
        image_label_list = []

        for EP_data_dir in EP_segmentation_list:
            EP_data = np.load(os.path.join(self.EP_folderpath, EP_data_dir))

            image_label = EP_data_dir.split("_")[2]

            EP_data_list.append(EP_data)
            image_label_list.append(image_label)

        return EP_data_list, image_label_list
    
Full_dataset = EP_dataset(EP_folderpath= "/media/ubuntu/sda/data/mouse6/output/04_get_neuron_channel/seg_test", image_folderpath="/media/ubuntu/sda/data/trigger/NaturalImages_new_2")
train_dataset, test_dataset = train_test_split(Full_dataset, test_size=0.2, random_state=42)

image_feature = pd.read_csv("/media/ubuntu/sda/data/mouse6/output/06_visual_decoding/image_feature.csv").iloc[:, 1:]
image_feature = torch.tensor(image_feature.to_numpy(), dtype=torch.float32)

Total Dataset Number: 6161
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000, 30)
(5000

- layer definition

In [3]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float()
                    * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(1)]
    
class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, dropout=0.1):
        super(DataEmbedding, self).__init__()
        
        self.value_embedding = nn.Linear(c_in, d_model)  

        self.position_embedding = PositionalEmbedding(d_model=d_model)
        self.dropout = nn.Dropout(p=dropout)
        self.mask_token = nn.Parameter(torch.randn(1, d_model)) 
        
    def forward(self, x, x_mark, mask=None):
        x = self.value_embedding(x)

        if x_mark is not None:
            x = x +  self.position_embedding(x)

        if mask is not None:
            x = x * (~mask.bool()) + self.mask_token * mask.float()

        return self.dropout(x)

In [4]:
class TriangularCausalMask():
    def __init__(self, B, L, device="cpu"):
        mask_shape = [B, 1, L, L]
        with torch.no_grad():
            self._mask = torch.triu(torch.ones(mask_shape, dtype=torch.bool), diagonal=1).to(device)

    @property
    def mask(self):
        return self._mask


class ProbMask():
    def __init__(self, B, H, L, index, scores, device="cpu"):
        _mask = torch.ones(L, scores.shape[-1], dtype=torch.bool).to(device).triu(1)
        _mask_ex = _mask[None, None, :].expand(B, H, L, scores.shape[-1])
        indicator = _mask_ex[torch.arange(B)[:, None, None],
                    torch.arange(H)[None, :, None],
                    index, :].to(device)
        self._mask = indicator.view(scores.shape).to(device)

    @property
    def mask(self):
        return self._mask
    

class EncoderLayer(nn.Module):
    def __init__(self, attention, d_model, d_ff=None, dropout=0.1, activation="relu"):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.attention = attention
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        new_x, attn = self.attention(
            x, x, x,
            attn_mask=attn_mask,
            tau=tau, delta=delta
        )
        x = x + self.dropout(new_x)

        y = x = self.norm1(x)
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))

        return self.norm2(x + y), attn
    
class Encoder(nn.Module):
    def __init__(self, attn_layers, conv_layers=None, norm_layer=None):
        super(Encoder, self).__init__()
        self.attn_layers = nn.ModuleList(attn_layers)
        self.conv_layers = nn.ModuleList(conv_layers) if conv_layers is not None else None
        self.norm = norm_layer

    def forward(self, x, attn_mask=None, tau=None, delta=None):
        # x [B, L, D]
        attns = []
        if self.conv_layers is not None:
            for i, (attn_layer, conv_layer) in enumerate(zip(self.attn_layers, self.conv_layers)):
                delta = delta if i == 0 else None
                x, attn = attn_layer(x, attn_mask=attn_mask, tau=tau, delta=delta)
                x = conv_layer(x)
                attns.append(attn)
            x, attn = self.attn_layers[-1](x, tau=tau, delta=None)
            attns.append(attn)
        else:
            for attn_layer in self.attn_layers:
                x, attn = attn_layer(x, attn_mask=attn_mask, tau=tau, delta=delta)
                attns.append(attn)

        if self.norm is not None:
            x = self.norm(x)

        return x, attns
    
class FullAttention(nn.Module):
    def __init__(self, mask_flag=True, factor=5, scale=None, attention_dropout=0.1, output_attention=False):
        super(FullAttention, self).__init__()
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, H, E = queries.shape
        _, S, _, D = values.shape
        scale = self.scale or 1. / math.sqrt(E)

        scores = torch.einsum("blhe,bshe->bhls", queries, keys)

        if self.mask_flag:
            if attn_mask is None:
                attn_mask = TriangularCausalMask(B, L, device=queries.device)

            scores.masked_fill_(attn_mask.mask, -np.inf)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        V = torch.einsum("bhls,bshd->blhd", A, values)

        if self.output_attention:
            return V.contiguous(), A
        else:
            return V.contiguous(), None
        
class AttentionLayer(nn.Module):
    def __init__(self, attention, d_model, n_heads, d_keys=None,
                 d_values=None):
        super(AttentionLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)
        d_values = d_values or (d_model // n_heads)

        self.inner_attention = attention
        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_model, d_keys * n_heads)
        self.value_projection = nn.Linear(d_model, d_values * n_heads)
        self.out_projection = nn.Linear(d_values * n_heads, d_model)
        self.n_heads = n_heads

    def forward(self, queries, keys, values, attn_mask, tau=None, delta=None):
        B, L, _ = queries.shape
        _, S, _ = keys.shape
        H = self.n_heads

        queries = self.query_projection(queries).view(B, L, H, -1)
        keys = self.key_projection(keys).view(B, S, H, -1)
        values = self.value_projection(values).view(B, S, H, -1)

        out, attn = self.inner_attention(
            queries,
            keys,
            values,
            attn_mask,
            tau=tau,
            delta=delta
        )
        out = out.view(B, L, -1)

        return self.out_projection(out), attn

- Model

In [5]:
class global_config:
    def __init__(self):
        self.task_name = 'classification'  # Example task name
        self.seq_len = 30                 # Sequence length
        self.pred_len = 250                # Prediction length
        self.output_attention = False      # Whether to output attention weights
        self.d_model = 250                 # Model dimension
        self.dropout = 0.3                # Dropout rate
        self.factor = 1                    # Attention scaling factor
        self.n_heads = 2                   # Number of attention heads
        self.e_layers = 1                  # Number of encoder layers
        self.d_ff = 64                    # Feedforward network dimension
        self.activation = 'gelu'           # Activation function
        self.enc_in = 30                   # Encoder input dimension (example value)

In [6]:
class iTransformer(nn.Module):
    def __init__(self, configs):
        super(iTransformer, self).__init__()
        self.task_name = configs.task_name
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.output_attention = configs.output_attention
        self.enc_embedding = DataEmbedding(configs.seq_len, configs.d_model, configs.dropout)
        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        FullAttention(False, configs.factor, attention_dropout=configs.dropout, output_attention=configs.output_attention),
                        configs.d_model, configs.n_heads
                    ),
                    configs.d_model,
                    configs.d_ff,
                    dropout=configs.dropout,
                    activation=configs.activation
                ) for l in range(configs.e_layers)
            ],
            norm_layer=torch.nn.LayerNorm(configs.d_model)
        )

    def forward(self, x_enc):
        enc_out = self.enc_embedding(x_enc, None)
        enc_out, attns = self.encoder(enc_out, attn_mask=None) 
        enc_out = enc_out[:, :30, :]      

        return enc_out

In [7]:
class PatchEmbedding(nn.Module):
    def __init__(self, emb_size=40):
        super().__init__()
        # Revised from ShallowNet
        self.tsconv = nn.Sequential(
            nn.Conv2d(1, 40, (1, 25), stride=(1, 1)),
            nn.AvgPool2d((1, 51), (1, 5)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.Conv2d(40, 40, (30, 1), stride=(1, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.Dropout(0.5),
        )

        self.projection = nn.Sequential(
            nn.Conv2d(40, emb_size, (1, 1), stride=(1, 1)),  
            Rearrange('b e (h) (w) -> b (h w) e'),
        )

    def forward(self, x: Tensor) -> Tensor:
        # b, _, _, _ = x.shape
        x = x.unsqueeze(1)     
        # print("x", x.shape)   
        x = self.tsconv(x)
        # print("tsconv", x.shape)   
        x = self.projection(x)
        # print("projection", x.shape)  
        return x

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FlattenHead(nn.Sequential):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        return x

class Enc_eeg(nn.Sequential):
    def __init__(self, emb_size=40, **kwargs):
        super().__init__(
            PatchEmbedding(emb_size),
            FlattenHead()
        )

class Proj_eeg(nn.Sequential):
    def __init__(self, embedding_dim=1440, proj_dim=1024, drop_proj=0.5):
        super().__init__(
            nn.Linear(embedding_dim, proj_dim),
            ResidualAdd(nn.Sequential(
                nn.GELU(),
                nn.Linear(proj_dim, proj_dim),
                nn.Dropout(drop_proj),
            )),
            nn.LayerNorm(proj_dim),
        )

In [8]:
class ClipLoss(nn.Module):
    def __init__(
        self,
        local_loss=False,
        gather_with_grad=False,
        cache_labels=False,
        rank=0,
        world_size=1,
        use_horovod=False,
    ):
        super().__init__()
        self.local_loss = local_loss
        self.gather_with_grad = gather_with_grad
        self.cache_labels = cache_labels
        self.rank = rank
        self.world_size = world_size
        self.use_horovod = use_horovod

        # cache state
        self.prev_num_logits = 0
        self.labels = {}

    def forward(self, image_features, logit_scale):
        device = image_features.device
        if self.world_size > 1:
            all_image_features = gather_features(
                image_features,
                None,
                self.local_loss,
                self.gather_with_grad,
                self.rank,
                self.world_size,
                self.use_horovod,
            )

            logits_per_image = logit_scale * all_image_features @ all_image_features.T
        else:
            logits_per_image = logit_scale * image_features @ image_features.T

        # calculated ground-truth and cache if enabled
        num_logits = logits_per_image.shape[0]
        if self.prev_num_logits != num_logits or device not in self.labels:
            labels = torch.arange(num_logits, device=device, dtype=torch.long)
            if self.world_size > 1 and self.local_loss:
                labels = labels + num_logits * self.rank
            if self.cache_labels:
                self.labels[device] = labels
                self.prev_num_logits = num_logits
        else:
            labels = self.labels[device]

        # Calculate loss
        loss = F.cross_entropy(logits_per_image, labels)

        # Find the predicted labels (highest probability)
        predicted_labels = torch.argmax(logits_per_image, dim=1)

        return loss, predicted_labels

In [9]:
class EP_encoder(nn.Module):    
    def __init__(self, global_config):
        super(EP_encoder, self).__init__()
        self.encoder = iTransformer(global_config)   
        self.enc_eeg = Enc_eeg()
        self.proj_eeg = Proj_eeg()        
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))
        self.loss_func = ClipLoss()       
         
    def forward(self, x):
        x = self.encoder(x)
        eeg_embedding = self.enc_eeg(x)
        out = self.proj_eeg(eeg_embedding)
        return out  

In [10]:

config = global_config() 
EP_encoder_v1 = EP_encoder(global_config = config)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    EP_encoder_v1 = nn.DataParallel(EP_encoder_v1)
EP_encoder_v1 = EP_encoder_v1.to(device='cuda')

In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [12]:
def train_model(encoder_modle, dataloader, optimizer, device, image_feature):
    encoder_modle.train()
    total_loss = 0
    correct = 0
    total = 0
    EP_feature_list = []
    predict_label = []
    actual_label = []
    image_feature = image_feature.to(device)
    for batch_id, (EP_data, label) in enumerate(dataloader):

        EP_data = EP_data.to(device)
        label = label.to(device)
        batch_size = EP_data.size(0)
        selected_image_features = image_feature[label - 1]

        optimizer.zero_grad()
        print(EP_data.shape)
        EP_feature = encoder_modle(EP_data).float()
        EP_feature_list.append(EP_feature)

        img_loss, min_label = encoder_modle.loss_func(EP_feature, selected_image_features, encoder_modle.logit_scale)
        img_loss.backward()

        optimizer.step()
        total_loss += img_loss.item()
        total += batch_size

        predict_label.append(min_label)
        actual_label.append(label)

    predict_label = torch.cat(predict_label, dim=0)
    actual_label = torch.cat(actual_label, dim=0)

    correct += (predict_label == actual_label).sum().item()

    average_loss = total_loss / (batch_id+1)
    accuracy = correct / total

    return average_loss, accuracy, torch.cat(EP_feature_list, dim=0)

In [13]:
def evaluate_model(encoder_modle, dataloader, device, image_feature):
    encoder_modle.eval()

    total_loss = 0
    correct = 0
    total = 0
    EP_feature_list = []
    predict_label = []
    actual_label = []
    image_feature = image_feature.to(device)
    
    with torch.no_grad():
        for batch_id, (EP_data, label) in enumerate(dataloader):

            EP_data = EP_data.to(device)
            label = label.to(device)
            selected_image_features = image_feature[label - 1]
            batch_size = EP_data.size(0)

            EP_feature = encoder_modle(EP_data).float()
            EP_feature_list.append(EP_feature)

            img_loss, min_label = encoder_modle.loss_func(EP_feature, selected_image_features, encoder_modle.logit_scale)            
            total_loss += img_loss.item()

            total += batch_size
    
            predict_label.append(min_label)
            actual_label.append(label)
    
        predict_label = torch.cat(predict_label, dim=0)
        actual_label = torch.cat(actual_label, dim=0)
    
        correct += (predict_label == actual_label).sum().item()
    
        average_loss = total_loss / (batch_id+1)
        accuracy = correct / total
        
        return average_loss, accuracy, predict_label, actual_label

In [14]:
import torch.optim as optim

optimizer = optim.Adam(EP_encoder_v1.parameters(), lr=0.0001)

def main_train_loop(date, encoder_modle, train_loader, test_loader, optimizer, device, config, image_feature):
    print('start train')
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []
    results = []

    for epoch in range(15):
        print(epoch + 1)
        train_loss, train_accuracy, features_tensor = train_model(encoder_modle, train_loader, optimizer, device, image_feature)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        test_loss, test_accuracy, test_predict, test_actual = evaluate_model(encoder_modle, test_loader, device, image_feature)
        
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)

        if (epoch +1) % 5 == 0:     
            print(f'Epoch: {epoch + 1}')
            os.makedirs(f"./models/'classification'/{date}", exist_ok=True)             
            file_path = f"./models/'classification'/{date}/{epoch+1}.pth"
            torch.save(encoder_modle.state_dict(), file_path)
        
            print(f"model saved in {file_path}!")
            print(f"train loss: {train_loss} at epoch: {epoch + 1}")
            print(f"train accuracy: {train_accuracy} at epoch: {epoch + 1}")
            print(f"test loss: {test_loss} at epoch: {epoch + 1}")
            print(f"test accuracy: {test_accuracy} at epoch: {epoch + 1}")
            print("----------")
            
        epoch_results = {
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "test_loss": test_loss,
        "test_accuracy": test_accuracy,
        "test_predict": test_predict,
        "test_actual": test_actual
        }
    
    results.append(epoch_results)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Loss curve
    axs[0].plot(train_losses, label='Train Loss')
    axs[0].plot(test_losses, label='Test Loss')
    axs[0].legend()
    axs[0].set_title("Loss Curve")

    # Overall accuracy curve
    axs[1].plot(train_accuracies, label='Train Accuracy')
    axs[1].plot(test_accuracies, label='Test Accuracy')
    axs[1].legend()
    axs[1].set_title("Accuracy Curve")

    plt.savefig(f'{date}_curve.png')

    return results

In [15]:
results = main_train_loop(encoder_modle=EP_encoder_v1, train_loader=train_loader, test_loader=test_loader, optimizer=optimizer, device='cuda', config=global_config,date='20250305', image_feature=image_feature)

start train
1
torch.Size([32, 5000, 30])


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.96 GiB. GPU 0 has a total capacity of 11.69 GiB of which 2.84 GiB is free. Including non-PyTorch memory, this process has 7.14 GiB memory in use. Of the allocated memory 6.93 GiB is allocated by PyTorch, and 29.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)